In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random

## Create Dataset

In [2]:
def generate_sentence(min_len=0, max_len=20, pos=True):
    if pos:
        n = np.random.randint(np.ceil(min_len / 3), np.ceil((max_len) / 3))
        sentence = n*"a" + n*"b" + n*"c"
        return sentence, len(sentence)
    else:
        n_0 = np.random.randint(0, max_len)
        n_1 = np.random.randint(0, max_len - n_0 + 1)
        n_2 = np.random.randint(min_len - n_0 - n_1, max_len - n_0 - n_1 + 1)
        sentence = n_0 * "a" + n_1 * "b" + n_2 * "c"
        return sentence, len(sentence)

def create_data(size=10000, balance=0.1, min_len=0, max_len=20, train=True):
    data = []
    sentence_lengths = []

    for i in range(int(size*balance)):
        sentence, sentence_length = generate_sentence(min_len=min_len, max_len=max_len, pos=True)
        while sentence == "aabbcc" and train:
            sentence, sentence_length = generate_sentence(min_len=min_len, max_len=max_len, pos=True)
        if not train:
            sentence, sentence_length = "aabbcc", 6
        data.append((sentence, 1))
        sentence_lengths.append(sentence_length)
    for i in range(int((size - (size*balance)))):
        sentence, sentence_length = generate_sentence(min_len=min_len, max_len=max_len, pos=False)
        data.append((sentence, 0))
        sentence_lengths.append(sentence_length)
    
    random.shuffle(data)
    average_length = sum(sentence_lengths) / len(sentence_lengths)
    return data, average_length

train_data, avg_sent_length_train = create_data(size=1000, balance=0.5, min_len=0, max_len=20, train=True)
val_data, avg_sent_length_val = create_data(size=1000, balance=0.5, min_len=0, max_len=20, train=False)
test_data, avg_sent_length_test = create_data(size=1000, balance=0.5, min_len=21, max_len=30, train=True)

print(f"Train Data Sample:\n{train_data}")
print(f"Average Sentence Length:\n{avg_sent_length_train}")
print(f"Val Data Sample:\n{val_data}")
print(f"Average Sentence Length:\n{avg_sent_length_val}")
print(f"Test Data Sample:\n{test_data}")
print(f"Average Sentence Length:\n{avg_sent_length_test}")


Train Data Sample:
[('', 1), ('', 1), ('aaabbbccc', 1), ('aaaaaaabbbbbbbbbb', 0), ('aaaabbbbcccc', 1), ('aaaaaabbbbbbcccccc', 1), ('aabccccccccccc', 0), ('aaaaabbbbbccccc', 1), ('aaaaaaaaabbbbbcccc', 0), ('aaaaaaaaaaaaaaaaaaab', 0), ('aaabbbccc', 1), ('aaaaaaaaaaaaabbbbbbb', 0), ('aaaabbbbcccc', 1), ('aaaaaabbbbbbcccccc', 1), ('aaaaabbbbbbbbb', 0), ('aaaaaaaaabbbbb', 0), ('abc', 1), ('aaaaabbbbbccccc', 1), ('aaaaabbbb', 0), ('aaaaaaaaabbbbbbbbb', 0), ('abc', 1), ('aaaaaabbbbbbcccccc', 1), ('aaaabbbcccc', 0), ('aaabbbccc', 1), ('aaaaaaaaaaaaaabbbb', 0), ('aaabbbbbbbbbbbbbccc', 0), ('aaaaaaaaaaaaaab', 0), ('aaaabbbbcccc', 1), ('aaaaaaaaaaaaaaaab', 0), ('aaaaabbbbbbbbb', 0), ('aaaaaaaabbbbbcccc', 0), ('bbbbbbbcccc', 0), ('abc', 1), ('aaaaaaaaaaaaaaaaaaab', 0), ('bbbbbbbb', 0), ('aaabbbccc', 1), ('abc', 1), ('aaaaaabbbbbbcccccc', 1), ('aaaaaaaaaaa', 0), ('bbbcccccccccccc', 0), ('aaabbbbbbbbbbbbbbbb', 0), ('aaaaabbbbbccccc', 1), ('aaaaabbbbbccccc', 1), ('aaaaabbbbbbbbbccc', 0), ('aaaaaaaaaa

In [3]:
# Dependencies
import torch
import torch.nn as nn
import torch.optim as optim

In [4]:
# Encoding data
char_to_index = {'a':[1,0,0], 'b':[0,1,0], 'c':[0,0,1]}
#index_to_char = {v: k for k, v in char_to_index.items()}

def creat_tensors(data, max_l):
    X = []
    y = []
    ml = 0

    for sent, label in data:
        X.append([char_to_index[char] for char in sent])
        y.append(label)

    # Padding to be able to convert to tensor
    X = [sent + [[0,0,0]] * (max_l - len(sent)) for sent in X]

    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.float32)

    return X, y

X_train, y_train = creat_tensors(train_data, max_l=30)
X_test, y_test = creat_tensors(test_data, max_l=30)
X_val, y_val = creat_tensors(val_data, max_l=30)

In [5]:
from torch.utils.data import TensorDataset, DataLoader

BATCH_SIZE = 64
train_set = TensorDataset(X_train, y_train)
val_set = TensorDataset(X_val, y_val)
testset = TensorDataset(X_test, y_test)

train_set, val_same_set = torch.utils.data.random_split(train_set, [900, 100])


train_loader = DataLoader(train_set, BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_set, BATCH_SIZE, shuffle=True)
val_same_loader = DataLoader(val_same_set, BATCH_SIZE, shuffle=True)
test_loader = DataLoader(testset, BATCH_SIZE, shuffle=True)

## Train and define model

In [6]:
# Set device
if torch.cuda.is_available():
    device = 'cuda:0'
elif torch.backends.mps.is_available():
    device = 'mps:0'
else:
    device = 'cpu'
print('GPU State:', device)

GPU State: cpu


In [7]:
class RNN(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, hidden_depth) -> None:
        super().__init__()

        self.lstm = nn.RNN(embedding_dim, hidden_dim, hidden_depth, batch_first=True)
        self.classification= nn.Sequential(
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        output, hn = self.lstm(x)
        logits = self.classification(output[:,-1,:])
        #print(logits.size())
        return torch.squeeze(logits)

In [8]:
input_size = X_train.shape[2]
hidden_size = 5
hidden_depth = 1
num_epochs = 200
lr=0.001

model = RNN(input_size, hidden_size, hidden_depth)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        #inputs = inputs.unsqueeze(-1)  # Add input_size dimension
        # Forward pass
        #print(inputs.shape)
        outputs = model(inputs)
        #print(outputs.shape)
        #print(outputs)
        loss = criterion(outputs, targets)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss / len(train_loader):.4f}')

# for param in model.parameters():
#     print(param5

Epoch [10/200], Loss: 0.6956
Epoch [20/200], Loss: 0.6921
Epoch [30/200], Loss: 0.6925
Epoch [40/200], Loss: 0.6914
Epoch [50/200], Loss: 0.3737
Epoch [60/200], Loss: 0.2754
Epoch [70/200], Loss: 0.2470
Epoch [80/200], Loss: 0.2332
Epoch [90/200], Loss: 0.2523
Epoch [100/200], Loss: 0.2455
Epoch [110/200], Loss: 0.2062
Epoch [120/200], Loss: 0.2081
Epoch [130/200], Loss: 0.1978
Epoch [140/200], Loss: 0.1906
Epoch [150/200], Loss: 0.1905
Epoch [160/200], Loss: 0.1974
Epoch [170/200], Loss: 0.1854
Epoch [180/200], Loss: 0.2039
Epoch [190/200], Loss: 0.1794
Epoch [200/200], Loss: 0.1723


In [9]:
from torcheval.metrics import BinaryF1Score

def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    metric = BinaryF1Score()

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            metric.update(pred, y)
            #print(pred)
            #print(y)
            #print("\n")

            test_loss += loss_fn(pred, y).item()
            correct += (torch.round(pred) == y).type(torch.float).sum().item()
            


    test_loss /= num_batches
    
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, F1-score: {metric.compute()}, Avg loss: {test_loss:>8f} \n")
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)


test_loop(val_loader, model, loss_fn)

Test Error: 
 Accuracy: 91.2%, F1-score: 0.9191176295280457, Avg loss: 0.261776 



In [10]:
test_loop(test_loader, model, loss_fn)

Test Error: 
 Accuracy: 72.9%, F1-score: 0.7107790112495422, Avg loss: 0.422946 



In [11]:
test_loop(val_same_loader, model, loss_fn)

Test Error: 
 Accuracy: 93.0%, F1-score: 0.9230769276618958, Avg loss: 0.238236 



## Parameter sweep

In [25]:
sweep_config = {
    'method': 'random',
}

metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'hidden_size': {
        'values': [3, 5, 10, 20]
    },
    'hidden_depth': {
        'values': [1, 2, 3]
    },
    'lr': {
        'values': [1e-2, 1e-3, 1e-4]
    },
    'num_epochs': {
        'values': [100, 200, 300]
    },
}

sweep_config['parameters'] = parameters_dict

import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'hidden_depth': {'values': [1, 2, 3]},
                'hidden_size': {'values': [3, 5, 10, 20]},
                'lr': {'values': [0.01, 0.001, 0.0001]},
                'num_epochs': {'values': [100, 200, 300]}}}


In [26]:
import wandb
sweep_id = wandb.sweep(sweep_config, project='lstm-sweep')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: uas5wibk
Sweep URL: https://wandb.ai/jarlku/lstm-sweep/sweeps/uas5wibk


In [27]:

def train_wandb(config=None):
    with wandb.init(config=config):
        config = wandb.config
        
        train_loader = DataLoader(train_set, BATCH_SIZE, shuffle=True)
        val_loader = DataLoader(val_set, BATCH_SIZE, shuffle=True)
        
        model = LSTM(input_size, config.hidden_size, config.hidden_depth)
        optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)
        criterion = nn.BCELoss()
        

        for epoch in range(config.num_epochs):
            model.train()
            epoch_loss = 0

            for batch_idx, (inputs, targets) in enumerate(train_loader):
                #inputs = inputs.unsqueeze(-1)  # Add input_size dimension
                # Forward pass
                outputs = model(inputs)
                #print(outputs)
                loss = criterion(outputs, targets)
                
                # Backward pass and optimization
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                epoch_loss += loss.item()
            
            wandb.log({'loss': epoch_loss / len(train_loader)})
            if (epoch + 1) % 10 == 0:
                print(f'Epoch [{epoch + 1}/{config.num_epochs}], Loss: {epoch_loss / len(train_loader):.4f}')
        
        metric = BinaryF1Score()
        for inputs, targets in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            wandb.log({'val_loss': loss})
            metric.update(outputs, targets)
            wandb.log({'val_f1': metric.compute()})
            
            

wandb.agent(sweep_id, train, count=5)

In [28]:
wandb.agent(sweep_id, train_wandb, count=5)

wandb: Agent Starting Run: afv4rl0c with config:
wandb: 	hidden_depth: 3
wandb: 	hidden_size: 20
wandb: 	lr: 0.001
wandb: 	num_epochs: 100
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jarlsoeren (jarlku). Use `wandb login --relogin` to force relogin


Epoch [10/100], Loss: 0.3160
Epoch [20/100], Loss: 0.3162


wandb: Ctrl + C detected. Stopping sweep.


Epoch [30/100], Loss: 0.3161
Epoch [40/100], Loss: 0.3161
Epoch [50/100], Loss: 0.3162